In [1]:
import xml.etree.ElementTree as ET
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import os
import re
from scipy.sparse import vstack

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\uralp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\uralp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
def preprocess(text):
    """
    Preprocess the text: tokenization, lowercasing, removing stopwords and punctuation.
    """
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(tokens)


In [4]:
def read_document(file_name):
    """
    Read and parse an XML document, extracting sentences.
    """
    with open(file_name, 'r', encoding='iso-8859-5') as file:
        content = file.read()
    
    sentences = re.findall(r'<sentence id="[^"]*">(.*?)</sentence>', content, re.DOTALL)
   
    # Strip leading and trailing whitespaces from each sentence
    sentences = [sentence.strip() for sentence in sentences]
    
    elements_to_remove = ['\n', '\n1']
    for element in elements_to_remove:
        sentences = [sentence.replace(element, "") for sentence in sentences]
    
    return sentences

In [5]:
def get_file_names(folder_path):
    """
    Get a list of XML file names in the given folder.
    """
    return [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.xml')]

In [6]:
def calculate_tfidf(corpus):
    """
    Calculate the TF-IDF matrix for the corpus.
    """
    vectorizer = TfidfVectorizer()
    return vectorizer, vectorizer.fit_transform(corpus)

In [7]:
def summarize_document(file_name, vectorizer, tfidf_matrix, doc_index):
    """
    Summarize the document specified by the file name.
    """
    sentences = read_document(file_name)
    preprocessed_sentences = [preprocess(sentence) for sentence in sentences]
    sentence_vectors = vectorizer.transform(preprocessed_sentences)

    sim_scores = []
    for i in range(len(sentences)):
        print(i)
        # Handle edge cases to avoid empty matrix
        if len(sentences) == 1:
            # If there's only one sentence, skip similarity calculation
            continue

        # Create a matrix excluding the current sentence
        if i == 0:
            matrix_without_sentence = tfidf_matrix[doc_index+1:]
        elif i == len(sentences) - 1:
            matrix_without_sentence = tfidf_matrix[:doc_index]
        else:
            matrix_without_sentence = vstack([tfidf_matrix[:doc_index], tfidf_matrix[doc_index+1:]])

        # Check if matrix_without_sentence is not empty
        if matrix_without_sentence.shape[0] > 0:
            sim = cosine_similarity(sentence_vectors[i], matrix_without_sentence)
            sim_scores.append(np.mean(sim))

    # Select top 5 sentences if there are enough sentences
    if len(sim_scores) > 0:
        top_indices = np.argsort(sim_scores)[-5:]
        summary = [sentences[i] for i in top_indices]
    else:
        summary = []

    for sentence in summary:
        print(sentence)

In [8]:
# Specify the folder containing the XML documents
folder_path = 'HW2_Data/Task2/'

In [9]:
# Read all documents and create a corpus
file_names = get_file_names(folder_path)
corpus = []
for file in file_names:
    corpus.extend(read_document(file))
    

In [10]:
# Calculate TF-IDF for the entire corpus
vectorizer, tfidf_matrix = calculate_tfidf(corpus)
print(tfidf_matrix)

  (0, 54674)	0.15903892594983834
  (0, 3791)	0.07811157963427807
  (0, 54644)	0.08570722989696063
  (0, 4368)	0.08915683076230238
  (0, 61613)	0.042084437580329294
  (0, 89189)	0.13037258052494585
  (0, 22338)	0.041779721981865456
  (0, 53915)	0.060526493705818064
  (0, 61930)	0.16146726899338937
  (0, 66644)	0.14312014186419272
  (0, 16924)	0.11962978508629077
  (0, 61363)	0.02400477218482631
  (0, 79034)	0.08121248752540744
  (0, 33572)	0.07672452506422192
  (0, 16345)	0.05814396365605151
  (0, 43246)	0.34298147705306514
  (0, 15572)	0.16043306337301327
  (0, 60022)	0.18659101992988994
  (0, 58201)	0.15257378053427398
  (0, 15375)	0.03120596410136165
  (0, 43944)	0.1117132062440983
  (0, 52270)	0.2956725987743083
  (0, 53294)	0.15258974613154305
  (0, 59689)	0.29347811512745137
  (0, 75257)	0.633026969300207
  :	:
  (835766, 65735)	0.12082088593336131
  (835766, 17482)	0.13530905987348824
  (835766, 16970)	0.0662835373930054
  (835766, 25643)	0.0659597919750884
  (835766, 29488)	0.05

In [11]:
# Summarize a specific document
document_to_summarize = "HW2_Data/Task2/06_1.xml"
doc_index = file_names.index(document_to_summarize)
summarize_document(document_to_summarize, vectorizer, tfidf_matrix, doc_index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
I also accept that this may become a matter about which questions might be asked if the orders sought by the applicants are made.
7 At least for those reasons, this Court has hel